# Drivers's performance - Braspress

## Libraries

In [369]:
import importlib
import numpy as np
import pandas as pd
import telemetry
import braspress

## Settings

In [395]:
settings = {
    "telemetry_dir": "Dataset/Telemetry Data",
    "braspress_branches_path": "Dataset/braspress_branches.csv",
    "braspress_valid_lines_path": "Dataset/valid_lines.csv",
    "dates": (
        (2024, 10),
        (2024, 11),
        (2024, 12)
    ),
    "min_displacement": 5.0,
    "braspress_proximity_threshold": 5.0,
    "avg_fuel_consumption_number_of_std": 2.0
}

**October 2024 Analysis**

Extract data

In [396]:
importlib.reload(telemetry)
oct2024 = telemetry.Telemetry(settings, 2024, 10, max_lines_to_read=1000)

In [397]:
drivers_stats = oct2024.get_driver_stats()
drivers_stats

AttributeError: 'list' object has no attribute 'groupby'

In [ ]:
importlib.reload(braspress)
import braspress

valid_lines_with_stats = braspress.Braspress(settings).extract_valid_lines_with_stats(drivers_stats)
valid_lines_with_stats

,avg_fuel_consumption
line_id,
ABSSAO,NaN
ABSTFD,NaN
AJUBHZ,NaN
AJUCPQ,NaN
AJUFSA,3.114934
...,...
XAPGPB,NaN
XAPITJ,NaN
XAPMGF,NaN


In [398]:
valid_lines_with_stats

,avg_fuel_consumption
line_id,
ABSSAO,NaN
ABSTFD,NaN
AJUBHZ,NaN
AJUCPQ,NaN
AJUFSA,3.114934
...,...
XAPGPB,NaN
XAPITJ,NaN
XAPMGF,NaN
